In [1]:
import pandas as pd
import numpy as np

In [3]:
%%time
#twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
twitter = pd.read_csv("before_balancing_data.csv") 
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores

Wall time: 3.65 s


In [4]:
twitter

,_score,tweet_id,text,identification,emotion
0,391,0x376b20,people who post add me on snapchat must be deh...,train,anticipation
1,433,0x2d5350,brianklaas as we see trump is dangerous to fre...,train,sadness
2,376,0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear
3,120,0x1d755c,riskshow thekevinallison thx for the best time...,train,joy
4,1021,0x2c91a8,still waiting on those supplies liscus,train,anticipation
...,...,...,...,...,...
1451432,94,0x321566,im so happy nowonder the name of this show hap...,train,joy
1451433,627,0x38959e,in every circumtance id like to be thankful to...,train,joy
1451434,274,0x2cbca6,theres currently two girls walking around the ...,train,joy
1451435,840,0x24faed,ah corporate life where you can date using ju...,train,joy


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

#Here we get the TF_IDF for the first 100 features:
number_features = 69000

tkzr = TweetTokenizer(strip_handles = True)

tfidf_vectorizer = TfidfVectorizer(
                            lowercase=True,
                            max_features= number_features,
                            #max_df=0.8,
                            #min_df=2,
                            #ngram_range = (1,3),
                            #stop_words = "english",
                            tokenizer = tkzr.tokenize
                            )

vectors = tfidf_vectorizer.fit_transform(twitter['text'])

feature_names = tfidf_vectorizer.get_feature_names()

In [6]:
# validate: split into train and test
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(vectors, twitter['emotion'], test_size=0.2, random_state=42)

# model
lr = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=1000,
                   multi_class='multinomial', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=2,
                   warm_start=False)

#the ones that first worked were -> solver: liblinear  ;  multi_class: auto

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)


print('Accuracy Score:', accuracy_score(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


convergence after 60 epochs took 83 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.4min finished


Accuracy Score: 0.5491201840930386


In [7]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [8]:
x_test_twitter = tfidf_vectorizer.transform(twitter_test_data['text'])

In [9]:
new_predict = lr.predict(x_test_twitter)

In [10]:
upload_df = pd.DataFrame(columns = [["id","emotion"]])
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = new_predict
upload_df

,id,emotion
0,0x28b412,trust
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,joy
4,0x26289a,trust
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,anticipation
411969,0x316b80,sadness
411970,0x29d0cb,joy


In [11]:
upload_df.to_csv("./uploads/logistic_try_2.csv",index=False)